In [ ]:
## import the necessary packages
## from skimage.measure import structural_similarity as ssim
## from skimage.metrics import structural_similarity as ssim
from skimage import measure
import matplotlib.pyplot as plt
import numpy as np
import cv2
import math
import os
import json
import time

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
def mse(imageA, imageB):
    # the 'Mean Squared Error' between the two images is the
    # sum of the squared difference between the two images;
    # NOTE: the two images must have the same dimension
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    
    err /= float(imageA.shape[0] * imageA.shape[1])

    # return the MSE, the lower the error, the more "similar"
    # the two images are
    return err

In [ ]:
def PSNR(img1, img2):
    
    mse = np.mean((img1 / 255. - img2 / 255.) ** 2)
    
    if mse == 0:
        
        return 100
    
    PIXEL_MAX = 1
    
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

## 批测试#### 

In [ ]:
def measures(groundtruth_dir_path, dehazy_dir_path, result_save_path, methods_name, SSIM = 1, PSNR_tag = 1, MSE = 1):

    
    gt_img_name = os.listdir(groundtruth_dir_path)
    
    num_img = len(gt_img_name)
    
    ssim_dict = {}
    
    psnr_dict = {}
    
    mse_dict = {}
    
    for gt_img in gt_img_name:
        
        gt_img_copy1 = gt_img
        
        img_form = gt_img_copy1.split(".",-1)[-1]
        
        gt_img_copy2 = gt_img
        
        gt_img_name = gt_img_copy2.rpartition('_')[0]
        
        # error :gt_img_name = gt_img_copy2.split("_",-1)[0]
        if methods_name != 'hazy':
            
            dehazy_img_name = gt_img_name + '_Dehaze_' + str(methods_name) + '.' + img_form
            
        else:
            
            dehazy_img_name = gt_img_name + '.' + img_form
            
        # print(dehazy_img_name) 
        ## get the full path of groudntruth and dehazy img
        gt_img_path = os.path.join(groundtruth_dir_path,gt_img)
        
        dehazy_img_path = os.path.join(os.path.join(dehazy_dir_path,methods_name),dehazy_img_name)
        #print(gt_img_path,dehazy_img_path)
        
        
        # calculate ssim psnr mse etc. And save the result to corresponding dict.
        gt_raw = cv2.imread(gt_img_path)
        
        dehazy_raw = cv2.imread(dehazy_img_path)
        
        if (SSIM == 1) or (MSE == 1):
            
            gt_gray = cv2.cvtColor(gt_raw, cv2.COLOR_BGR2GRAY)
            
            dehazy_gray = cv2.cvtColor(dehazy_raw, cv2.COLOR_BGR2GRAY)
            
            if SSIM == 1:
                
                ssim = measure.compare_ssim(gt_gray, dehazy_gray)
                
                ssim_dict[dehazy_img_name] = ssim
            
            if MSE == 1:
                
                m = mse(gt_gray, dehazy_gray)
                
                mse_dict[dehazy_img_name] = m
                
        if PSNR_tag == 1:
            
            psnr = PSNR(gt_raw, dehazy_raw)
            
            psnr_dict[dehazy_img_name] = psnr
            
    ## out of the loop
    if SSIM == 1:
        
        mean_ssim = sum(ssim_dict.values())/float(num_img)
        
        ssim_evaluate_file_name = 'SSIM_' + str(methods_name)+ '.txt'
        
        ssim_evaluate_path = os.path.join(result_save_path, ssim_evaluate_file_name)
        
        with open(ssim_evaluate_path, 'w') as f:
            f.write('TOTAL IMAGE NUMBER:')
            f.write('                   ')
            f.write(str(num_img))
            f.write('\n')
            f.write('MEAN SSIM:         ')
            f.write('                   ')
            f.write(str(mean_ssim))
            f.write('\n')
            f.write('\n')
            f.write('DETAILS AS FOLLOWS:')
            f.write('\n')
            f.write('              IMAGE              ')
            f.write('                 SSIM               ')            
            f.write('\n')
            for k in ssim_dict:
                f.write(str(k))
                f.write('    ')
                f.write(str(ssim_dict[k]))
                f.write('\n')
            f.write('\n')
            f.write('THAT\'s ALL !')
        
    if PSNR_tag == 1:
        
        mean_psnr = sum(psnr_dict.values())/float(num_img)
        
        psnr_evaluate_file_name = 'PSNR_' + str(methods_name)+ '.txt'
        
        psnr_evaluate_path = os.path.join(result_save_path, psnr_evaluate_file_name)
        
        with open(psnr_evaluate_path, 'w') as f:
            f.write('TOTAL IMAGE NUMBER:')
            f.write('                   ')
            f.write(str(num_img))
            f.write('\n')
            f.write('MEAN PSNR:         ')
            f.write('                   ')
            f.write(str(mean_psnr))
            f.write('\n')
            f.write('\n')
            f.write('DETAILS AS FOLLOWS:')
            f.write('\n')
            f.write('              IMAGE              ')
            f.write('                 PSNR               ')            
            f.write('\n')
            for k in psnr_dict:
                f.write(str(k))
                f.write('    ')
                f.write(str(psnr_dict[k]))
                f.write('\n')
            f.write('\n')
         
    if MSE == 1:
        
        mean_mse = sum(mse_dict.values())/float(num_img)
        
        mse_evaluate_file_name = 'MSE_' + str(methods_name)+ '.txt'
        
        mse_evaluate_path = os.path.join(result_save_path, mse_evaluate_file_name)
        
        with open(mse_evaluate_path, 'w') as f:
            f.write('TOTAL IMAGE NUMBER:')
            f.write('                   ')
            f.write(str(num_img))
            f.write('\n')
            f.write('MEAN MSE:         ')
            f.write('                   ')
            f.write(str(mean_mse))
            f.write('\n')
            f.write('\n')
            f.write('DETAILS AS FOLLOWS:')
            f.write('\n')
            f.write('              IMAGE              ')
            f.write('                 MSE               ')            
            f.write('\n')
            for k in mse_dict:
                f.write(str(k))
                f.write('    ')
                f.write(str(mse_dict[k]))
                f.write('\n')
            f.write('\n')
         
    summary_file_name = 'summary.txt'
    
    summary_file_path = os.path.join(result_save_path, summary_file_name)
    
    with open(summary_file_path, 'a+') as f:
        f.write("SUMMARY...")
        f.write('\n')
        f.write('\n')
        f.write('METHODS:'+'      '+ str(methods_name)+ '\n')
        f.write('TOTAL ' +str(num_img)+' IMAGES\n')
        f.write('MEAN SSIM:'+'      '+str(mean_ssim)+'\n')
        f.write('MEAN PSNR:'+'      '+str(mean_psnr)+'\n')
        f.write('MEAN MSE :'+'      '+str(mean_mse)+'\n')
        f.write('\n')
        f.write('\n')
        f.write('-------------------------------------------------------------------\n')
    
    print('methods: %s fine'%methods_name)

In [ ]:
def measures_all(groundtruth_dir_path, dehazy_dir, result_save_path, methods_list):
    
    begin = time.time()
    
    for methods in methods_list:
        
        measures(groundtruth_dir_path, dehazy_dir, result_save_path,methods)
        
    end = time.time()
    
    t = end - begin
    
    print("Work Done ! Using time: %s s" %t)
    

# 说明


In [ ]:
if __name__ == "__main__":
    # methods_list = ['hazy','proposed','ATM','BCCR','CAP','DCP','DehazeNet','FVR','MSCNN','NLD']
    
    methods_list = ['hazy','proposed','ATM','BCCR','CAP','DCP','DehazeNet','FVR','MSCNN','NLD']
    
    groundtruth_dir_path = './evaluate/groundtruth'
    
    dehazy_dir = './evaluate'
    
    result_save_path = './evaluate/result'
    
    measures_all('./evaluate/groundtruth', './evaluate', './evaluate/result', methods_list)